## Постановка задачи

Во четвертом модуле нас ждут две более сложные задачи: обработка списка поисковых запросов и статистики рекламных кампаний. 

Такие запросы можно использовать для настройки контекстной рекламы, и они в зависимости от сложности кампании могут быть весьма объемными. Представим, что вам в наследство от предыдущего работника достался файл с запросами и вам надо посчитать статистику в этом файле. 

Начнем с первой задачи: дана статистика поисковых запросов вашей потенциальной целевой аудитории за 11 дней. Необходимо посчитать распределение количества слов в поисковых запросах этого файла на тему недвижимости. Т. е. понять сколько запросов содержали 1 слово, 2 слова итд.

## Количество запросов в день
В этом шаге мы сначала проделаем простую операцию – посчитаем среднее количество запросов за 1 день в нашем файле. Посмотрим что есть в наших данных.

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 10)

In [2]:
data = pd.read_csv('./module04_files/keywords.csv')

data.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


Есть только общее число запросов за весь период. Однако, работать со статистикой за 11 дней не всегда удобно, хочется представлять, сколько запросов было за один день.

Давайте заведем новый столбец daily_shows, в котором выведем примерное количество запросов за 1 день (то есть поделим количество запросов за временной период на количество дней). Синтаксис будет простым:

In [3]:
data['daily_shows'] = data['shows'] / 11
data['ratio'] = round(data['daily_shows'] / data['shows'], 2)
data.head()

,keyword,shows,daily_shows,ratio
0,вк,64292779,5.844798e+06,0.09
1,одноклассники,63810309,5.800937e+06,0.09
2,порно,41747114,3.795192e+06,0.09
3,ютуб,39995567,3.635961e+06,0.09
4,вконтакте,21014195,1.910381e+06,0.09


In [4]:
data.drop(['ratio'], axis=1, inplace=True)

In [5]:
data = data[data['keyword'].str.contains('недвиж', case=False)]

In [6]:
data

,keyword,shows,daily_shows
2806,циан недвижимость москва,78729,7157.181818
3054,авито недвижимость,74037,6730.636364
3066,недвижимость,72984,6634.909091
4273,е1 недвижимость,56277,5116.090909
4912,нгс недвижимость новосибирск,54149,4922.636364
...,...,...,...
97433,авито санкт петербурге недвижимость,3720,338.181818
97674,емлс недвижимость,3781,343.727273
98902,авито рязань недвижимость,9734,884.909091
99002,недвижимость сочи,5097,463.363636


## ЗАМЕЧАНИЕ ПРИ РАСЧЕТАХ СТОЛБЦОВ
Иногда при расчете новых столбцов вы будете видеть предупреждение: 'A value is trying to be set on a copy of a slice from a DataFrame'. Для того, чтобы его избежать, используйте метод loc. Метод loc позволяет явно указать каким ячейкам мы назначаем новые значения (т. е. берем все строки (знак двоеточия) и столбец 'split').

Т. е. вместо 
```python
data['split'] = data['keyword'].str.split(' ')
```
явно указывайте на какие строки и столбцы вы вычисляете новые значения:
```python
data.loc[:, 'split'] = data['keyword'].str.split(' ')
```

Далее считаем кол-во слов в запросе:

In [8]:
data.loc[:, 'split'] = data['keyword'].str.split(' ')
data.head()

,keyword,shows,daily_shows,split
2806,циан недвижимость москва,78729,7157.181818,"[циан, недвижимость, москва]"
3054,авито недвижимость,74037,6730.636364,"[авито, недвижимость]"
3066,недвижимость,72984,6634.909091,[недвижимость]
4273,е1 недвижимость,56277,5116.090909,"[е1, недвижимость]"
4912,нгс недвижимость новосибирск,54149,4922.636364,"[нгс, недвижимость, новосибирск]"


In [10]:
data.apply(lambda x: len(x['split']), axis=1)

2806     3
3054     2
3066     1
4273     2
4912     3
        ..
97433    4
97674    2
98902    3
99002    2
99532    4
Length: 142, dtype: int64

## Упражнение
(1 возможный балл)
Представим, что вам надо посчитать показатель дневных просмотров в целых числах.

Примените функцию int к столбцу daily_shows. Можно не заводить новый столбец, а перезаписать существующий.

Какое значение daily_shows получится у поискового запроса 'недвижимость'? (В нашей версии датафрейма это третья сверху строка с индексом 3066)

In [23]:
#data.apply(lambda row: int(row['daily_shows']), axis=1)
#плохо

In [25]:
data['daily_shows'] = data['daily_shows'].astype('int64')
data['word_count'] = data['split'].apply(lambda x: len(x))
data.head()

,keyword,shows,daily_shows,split,word_count
2806,циан недвижимость москва,78729,7157,"[циан, недвижимость, москва]",3
3054,авито недвижимость,74037,6730,"[авито, недвижимость]",2
3066,недвижимость,72984,6634,[недвижимость],1
4273,е1 недвижимость,56277,5116,"[е1, недвижимость]",2
4912,нгс недвижимость новосибирск,54149,4922,"[нгс, недвижимость, новосибирск]",3


In [26]:
data['word_count'].value_counts()

3    72
2    25
4    20
5    12
6     7
8     3
9     1
7     1
1     1
Name: word_count, dtype: int64